In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [ ]:
def extract_result():
    # Khởi tạo trình duyệt Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # Mở trang web
    url = 'https://www.premierleague.com/results'
    driver.get(url)

    # Đặt thời gian chờ để cuộn trang
    pause_time = 3

    # Cuộn trang nhiều lần để tải hết dữ liệu
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Cuộn xuống cuối trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Chờ cho nội dung tải thêm
        time.sleep(pause_time)

        # Lấy chiều cao mới của trang
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Nếu chiều cao không thay đổi, đã cuộn hết trang
        if new_height == last_height:
            break

        last_height = new_height

    # Lấy nội dung HTML sau khi cuộn
    html_content = driver.page_source

    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    raw_data = pd.DataFrame(columns=['match_code', 'date', 'home', 'away', 'match_score'])
    doc = soup.find_all('div', class_='fixtures__date-container')

    for i in doc:
        dates = i.find('time', class_='fixtures__date fixtures__date--short')
        matches = i.find_all('li', class_='match-fixture')

        for match in matches:
            match_id = match.get('data-comp-match-item', 'NaN')
            home_team = match.get('data-home', 'NaN')
            away_team = match.get('data-away', 'NaN')
            score = match.find('span', class_='match-fixture__score').text if match.find('span', class_='match-fixture__score') else 'NaN'
        
            date = dates.text.strip() if dates else 'NaN'
            
            data = {
                'match_code': match_id,
                'date': date,
                'home': home_team,
                'away': away_team,
                'match_score': score,
            }
            raw_data.loc[len(raw_data)] = data

    driver.quit()
    raw_data.to_csv("data/result.csv", index=False, mode='a', header=False)

# Gọi hàm
extract_result()


In [3]:
# Tạo danh sách các câu lạc bộ với ID tương ứng
def extract_player():
    clubs = [
        (1, 'Arsenal'),
        (2, 'Aston-Villa'),
        (127, 'Bournemouth'),
        (130, 'Brentford'),
        (131, 'Brighton-and-Hove-Albion'),
        (4, 'Chelsea'),
        (6, 'Crystal-Palace'),
        (7, 'Everton'),
        (34, 'Fulham'),
        (8, 'Ipswich-Town'),
        (26, 'Leicester-City'),
        (10, 'Liverpool'),
        (11, 'Manchester-City'),
        (12, 'Manchester-United'),
        (13, 'Newcastle-United'),
        (15, 'Nottingham-Forest'),
        (20, 'Southampton'),
        (21, 'Tottenham-Hotspur'),
        (25, 'West-Ham-United'),
        (38, 'Wolverhampton-Wanderers')
    ]

    # Danh sách để lưu dữ liệu cầu thủ
    players_data = []

    # Khởi động WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for club_id, club_name in clubs:
        url = f'https://www.premierleague.com/clubs/{club_id}/{club_name}/squad?se=719'
        driver.get(url)

        # Lấy nội dung HTML sau khi cuộn
        html_content = driver.page_source

        # Sử dụng BeautifulSoup để phân tích HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        squad = soup.find_all('li', class_='stats-card')
        
        for player in squad:
            info = player.find('div', class_='stats-card__player-info')
            # Khai thác thông tin cầu thủ với xử lý lỗi
            first_name = info.find('div', class_='stats-card__player-first').text.strip() if info.find('div', class_='stats-card__player-first') else 'NaN'
            last_name = info.find('div', class_='stats-card__player-last').text.strip() if info.find('div', class_='stats-card__player-last') else 'NaN'
            position = info.find('div', class_='stats-card__player-position').text.strip() if info.find('div', class_='stats-card__player-position') else 'NaN'
            try:
                href = player.find('a')['href']
                player_id = href.split('/')[2]
            except:
                player_id = 'NaN'
            # Thêm thông tin cầu thủ vào danh sách
            players_data.append({
                'player_id': player_id,
                'first_name': first_name,
                'last_name': last_name,
                'position': position,
                'club': club_name
            })

    # Tạo DataFrame từ danh sách
    players_df = pd.DataFrame(players_data)

    # Lưu DataFrame vào file CSV
    players_df.to_csv('data/player.csv', index=False,header=False)

    # Đóng trình duyệt
    driver.quit()

# Gọi hàm để thực thi
extract_player()


In [ ]:
import re
def extract_goal():
    df = pd.read_csv('data/result.csv')
    raw_data = pd.DataFrame(columns=['player_id', 'time', 'match_code','date'])
    # Tạo danh sách để lưu trữ dữ liệu
    all_data = []

    # Duyệt qua từng dòng trong DataFrame
    n = df.shape[0]

    for i in range(n):
        match_id = df['match_code'][i]
        
        url = f'https://www.premierleague.com/match/{match_id}'

        # Khởi động trình duyệt Chrome
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)

        # Lấy nội dung HTML
        html_content = driver.page_source
        

        # Phân tích nội dung HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Lấy tên đội
        summary = soup.find('div', class_='mc-summary__scorebox-container')
        team_containers = summary.find_all('div', class_='mc-summary__team-container')
        teams = [container.find('span', class_='mc-summary__team-name u-hide-phablet').text for container in team_containers]

        # Hàm để trích xuất sự kiện cho một đội
        def extract_events(container, team_name, match_id):
            events = container.find_all('div', class_='mc-summary__event')
            for event in events:
                first = event.find('div', class_='mc-summary__scorer-name-first')
                last = event.find('div', class_='mc-summary__scorer-name-last')
                time = event.find('span', class_='mc-summary__event-time')
                href = event.find('a')['href']
                player_id = href.split('/')[2]

                event_data = {
                    'player_id': player_id,
                    'time': re.sub(r'\s+', ' ', time.text.strip().replace('\n', '')).replace(',', '-') if time else '',
                    'match_code': match_id,
                    'date': df['date'][i]
                }
                raw_data.loc[len(raw_data)] = event_data

        # Trích xuất sự kiện cho đội nhà
        home_container = soup.find('div', class_='matchEvents matchEventsContainer home')
        extract_events(home_container, teams[0],match_id)

        # Trích xuất sự kiện cho đội khách
        away_container = soup.find('div', class_='matchEvents matchEventsContainer away')
        extract_events(away_container, teams[1],match_id)
    driver.quit()  # Đóng trình duyệt sau khi lấy nội dung    
    raw_data.to_csv("data/goal.csv", index=False, mode='a')


In [15]:
extract_goal()

In [ ]:
def extract_appearance(df):
    # Tạo danh sách để lưu trữ dữ liệu
    all_data = []

    # Duyệt qua từng dòng trong DataFrame
    n = df.shape[0]

    for i in range(n):
        match_id = df['match_code'][i]
        url = f'https://www.premierleague.com/match/{match_id}'

        # Khởi động trình duyệt Chrome
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)

        # Lấy nội dung HTML
        html_content = driver.page_source
        

        # Phân tích nội dung HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        
        teams = soup.find_all('div',class_= 'matchLineupTeamContainer')
        
        for team in teams:
            stats= team.find_all('li',class_='player')
            for player in stats:
                href = player.find('a')['href']
                player_id = href.split('/')[2]
                
                data = {
                    'match_code': match_id,
                    'player_id': player_id,
                    'date': df['date'][i]
                }
                all_data.append(data)
    raw_data = pd.DataFrame(all_data)
    raw_data.to_csv("data/appearance.csv", index=False, mode='a',header=False)
    driver.quit()
    
    return raw_data
extract_appearance()

In [6]:
import pandas as pd
df = pd.read_csv('data/goal.csv')
df.drop(columns=['first_name','last_name','team']).to_csv('temp/goal.csv',index= False)


In [11]:
df = pd.read_csv('data/result.csv')
df.drop(columns='stadium').to_csv('temp/result.csv',index=False)

In [42]:
import pandas as pd
team = pd.read_csv('data/teams.csv')
#team
re = pd.read_csv('data/result.csv')

team.rename(columns={'team':'home'},inplace = True)

re = pd.merge(re,team,on= 'home')

re.drop(columns=' abbreviated',inplace=True)

In [45]:
re.columns.to_list()

['match_code',
 'date',
 'home',
 'away',
 'match_score',
 'stadium',
 'city',
 'capacity']

In [19]:
import pandas as pd
player = pd.read_csv('data/player.csv')
#team
goal = pd.read_csv('data/goal.csv')

In [20]:
df = pd.merge(goal,player,on = 'player_id')

In [24]:
df

,player_id,time,match_code,date,first_name,last_name,position,club
0,3756,65’,115903,Tue 22 Oct 2024,Chris,Wood,Forward,Nottingham-Forest
1,35670,07’,115906,Sun 20 Oct 2024,Jørgen,Strand Larsen,Forward,Wolverhampton-Wanderers
2,67178,33’,115906,Sun 20 Oct 2024,Josko,Gvardiol,Defender,Manchester-City
3,4505,90 +5’,115906,Sun 20 Oct 2024,John,Stones,Defender,Manchester-City
4,5178,29’ (pen),115900,Sun 20 Oct 2024,Mohamed,Salah,Forward,Liverpool
...,...,...,...,...,...,...,...,...
195,3756,23’,115832,Sat 17 Aug 2024,Chris,Wood,Forward,Nottingham-Forest
196,25474,86’,115832,Sat 17 Aug 2024,Antoine,Semenyo,Forward,Bournemouth
197,55776,37’ (pen),115833,Sat 17 Aug 2024,Lucas,Paquetá,Midfielder,West-Ham-United
198,108096,04’,115833,Sat 17 Aug 2024,Amadou,Onana,Midfielder,Aston-Villa


In [ ]:
df = pd.read_csv('data/result.csv')
raw_data = pd.DataFrame()
    # Tạo danh sách để lưu trữ dữ liệu
all_data = []

    # Duyệt qua từng dòng trong DataFrame
n = df.shape[0]

for i in range(1):
    match_id = df['match_code'][i]
    date = df['date'][i]
    
    url = f'https://www.premierleague.com/match/{match_id}'
    #url = 'https://www.premierleague.com/match/115873'
        # Khởi động trình duyệt Chrome
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)

        # Lấy nội dung HTML
    html_content = driver.page_source
        
        # Phân tích nội dung HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        event_line = soup.find('div', class_="eventLine timeLineEventsContainer is-completed")
        event_away = event_line.find_all('ul', class_="event__icons event__icons--away")
        event_home = event_line.find_all('ul', class_="event__icons event__icons--home")

        for i in event_away:
            # Lặp qua từng <ul> và tìm các div chứa thông tin sự kiện
            b = i.find_all('div', class_="eventInfo")
            
            for info in b:  # Duyệt qua từng sự kiện trong danh sách b
                # Kiểm tra xem có thông tin Yellow Card hay không
                if "Yellow Card" in info.text:
                    player_name = info.find('a', class_='name').text.strip()

                    # Lấy ID cầu thủ từ href
                    player_id = info.find('a', class_='name')['href'].split('/')[2]
                    minute = info.find('time', class_='min').text.strip()

            
                    data = {
                        'player_id': player_id,
                        'match_id': match_id,
                        'minute': minute,
                        'date': date,
                        'type': "yellow"
                    }
                    all_data.append(data)
                if "Red Card" in info.text:
                    player_name = info.find('a', class_='name').text.strip()

                    # Lấy ID cầu thủ từ href
                    player_id = info.find('a', class_='name')['href'].split('/')[2]

            
                    data = {
                        'player_id': player_id,
                        'match_id': match_id,
                        'date': date,
                        'type': "red"
                    }
                    all_data.append(data)
                    
        for i in event_home:
            # Lặp qua từng <ul> và tìm các div chứa thông tin sự kiện
            b = i.find_all('div', class_="eventInfo")
            
            for info in b:  # Duyệt qua từng sự kiện trong danh sách b
                # Kiểm tra xem có thông tin Yellow Card hay không
                if "Yellow Card" in info.text:
                    player_name = info.find('a', class_='name').text.strip()

                    # Lấy ID cầu thủ từ href
                    player_id = info.find('a', class_='name')['href'].split('/')[2]
                    
                    minute = info.find('time', class_='min').text.strip()

                    
                    data = {
                        'player_id': player_id,
                        'match_id': match_id,
                        'time': minute,
                        'date': date,
                        'type': 'yellow'
                    }
                    all_data.append(data)
                if "Red Card" in info.text:
                    player_name = info.find('a', class_='name').text.strip()

                    # Lấy ID cầu thủ từ href
                    player_id = info.find('a', class_='name')['href'].split('/')[2]

            
                    data = {
                        'player_id': player_id,
                        'match_id': match_id,
                        'date': date,
                        'type': "red"
                    }
                    all_data.append(data)
    except:
        continue
    raw_data = pd.DataFrame(all_data)
    
driver.quit()
raw_data.to_csv("data/cards.csv", index=False, mode='a',header=False)

In [ ]:
import pandas as pd

df = pd.read_csv("data/result.csv")
a = '2024-08-17'
raw_data[pd.to_datetime(raw_data['date']) == a]

,match_code,date,home,away,match_score
73,115827,Sat 17 Aug 2024,Man Utd,Fulham,1-0
74,115830,Sat 17 Aug 2024,Ipswich,Liverpool,0-2
75,115828,Sat 17 Aug 2024,Arsenal,Wolves,2-0
76,115829,Sat 17 Aug 2024,Everton,Brighton,0-3
77,115831,Sat 17 Aug 2024,Newcastle,Southampton,1-0
78,115832,Sat 17 Aug 2024,Nott'm Forest,Bournemouth,1-1
79,115833,Sat 17 Aug 2024,West Ham,Aston Villa,1-2
